In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/catboost_model'

In [17]:
num_users=5000
eye=np.eye(num_users)
def onehot_encode(x):
    return eye[x]

In [18]:
def gen_xy(sequence):
    """
    Returns
    ---
    x: (num_samples, 5)
    y: (num_samples,) 1-d vec for labels
    """

    x, y = [], []
    for i in range(len(sequence) - 2):
        x.append([*onehot_encode(sequence[i][0]), *onehot_encode(sequence[i][1]), *onehot_encode(sequence[i+1][0]), *onehot_encode(sequence[i+1][1]), *onehot_encode(sequence[i][2])])
        y.append(sequence[i + 1][-1])

    x = np.array(x)
    # x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    # x = x[:, :5]
    y = np.array(y)

    # if len(y.shape) > 1:
    #     y = y[:, -1]

    return x, y

sequence = pd.read_pickle("../data/data_100000_distr.pkl")[
    ["from_user_id", "to_user_id", "label"]
].values

x, y=gen_xy(sequence)

x.shape
y.shape

KeyboardInterrupt: 

In [ ]:
train_size=0.7
val_size=0.1

split1 = int(len(x) * train_size)
split2 = int(len(sequence) * (train_size + val_size))

x_train, y_train = x[:split1], y[:split1]
x_val, y_val = x[split1:split2], y[split1:split2]
x_test, y_test = x[split2:], y[split2:]

print(f"Trainset:\tx-{x_train.shape}\ty-{y_train.shape}")
print(f"Valset:  \tx-{x_val.shape}  \ty-{y_val.shape}")
print(f"Testset:\tx-{x_test.shape}\ty-{y_test.shape}")

In [ ]:
from sklearn.svm import SVC

svc=SVC()
svc.fit(x_train, y_train)

y_pred=svc.predict(x_test)
accuracy_score(y_test, y_pred)